# Import python libraries

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import  GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, plot_confusion_matrix

from sklearn.ensemble import BaggingClassifier 
# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### Import Train and Test Data

In [2]:
X_train= pd.read_csv("./datasets/X_train.csv" , squeeze = True)
y_train= pd.read_csv("./datasets/y_train.csv", squeeze = True)

X_test= pd.read_csv("./datasets/X_test.csv" , squeeze = True)
y_test= pd.read_csv("./datasets/y_test.csv", squeeze = True)

### Model 4: Bagged Decision Tree : Pipeline - GridSearch

In [3]:
# Let's set a pipeline up with two stages:
# 1. CountVectorizer (transformer)
# 2. Bagging Classifier (estimator)
pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('bc', BaggingClassifier())
])

In [4]:
# Search over the following values of hyperparameters:
# Maximum number of features fit: 2000, 3000, 4000, 5000
# Minimum number of documents needed to include token: 2, 3
# Maximum number of documents needed to include token: 90%, 95%
# Check (individual tokens) and also check (individual tokens and 2-grams).
pipe_params = {
    'cvec__max_features' : [200, 300, 400, 500],
    'cvec__stop_words' : [None,['Sherlock', 'Poirot'], 'english'],
    'cvec__ngram_range' : [(1,1), (1,2)],
    'bc__n_estimators':[1,3,5,10],
    'bc__max_features':[1,5,10]
}

In [5]:
# Instantiate GridSearchCV.
gs_bc = GridSearchCV(pipe,
                      pipe_params,
                      cv = 5,
                      verbose=1,
                      n_jobs=8)

In [6]:
# Fit GridSearch to training data.
gs_bc.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 312 tasks      | elapsed:    4.5s
[Parallel(n_jobs=8)]: Done 812 tasks      | elapsed:   10.1s
[Parallel(n_jobs=8)]: Done 1440 out of 1440 | elapsed:   17.4s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('bc', BaggingClassifier())]),
             n_jobs=8,
             param_grid={'bc__max_features': [1, 5, 10],
                         'bc__n_estimators': [1, 3, 5, 10],
                         'cvec__max_features': [200, 300, 400, 500],
                         'cvec__ngram_range': [(1, 1), (1, 2)],
                         'cvec__stop_words': [None, ['Sherlock', 'Poirot'],
                                              'english']},
             verbose=1)

In [7]:
gs_bc_df = pd.DataFrame(gs_bc.cv_results_)

In [8]:
gs_bc.best_estimator_

Pipeline(steps=[('cvec',
                 CountVectorizer(max_features=200, stop_words='english')),
                ('bc', BaggingClassifier(max_features=10))])

In [9]:
gs_bc.best_params_

{'bc__max_features': 10,
 'bc__n_estimators': 10,
 'cvec__max_features': 200,
 'cvec__ngram_range': (1, 1),
 'cvec__stop_words': 'english'}

### Run the model on Test data

In [10]:
# Evaluate the best fit model on the test data.
best_nb = gs_bc.best_estimator_
print(f"Training Score from best Bagged Decision Tree Classifier: {gs_bc.score(X_train , y_train)}")
print(f"Test Score from best Bagged Decision Tree Classifier: {gs_bc.score(X_test, y_test)}") 

Training Score from best Bagged Decision Tree Classifier: 0.8628048780487805
Test Score from best Bagged Decision Tree Classifier: 0.7519083969465649
